In [ ]:
import pandas as pd
import forecast_evaluation as fe

# Loading the data
Load the data with the `ForecastData()` class, and apply filters using the `.filter()` method

In [ ]:
data = fe.ForecastData(load_fer=True)
data.filter(variables=["gdpkp", "cpisa", "aweagg"])

### Loading data with extra labelling columns

In [ ]:
# get some new forecasts and a label
sample_forecasts = fe.create_sample_forecasts()
sample_forecasts["extra label"] = "Model family A"

# create new ForecastData with extra column
data_example_extra_columns = fe.ForecastData(load_fer=True)
data_example_extra_columns.add_forecasts(sample_forecasts, extra_ids=["extra label"])

# display new data added
data_example_extra_columns.df.tail()

# display original data
data_example_extra_columns.df.head()

# __Visualisations__

## Plot recent forecast errors against historical distribution of forecast errors

In [ ]:
# Example: highlight dates from 2022 to 2024
dates_to_highlight = pd.date_range(start="2022-01-01", end="2024-12-31", freq="QE")

plot = fe.plot_forecast_error_density(
    data=data,
    horizon=4,
    variable="cpisa",
    metric="yoy",
    frequency="Q",
    source="mpr",
    k=12,
    highlight_dates=dates_to_highlight,
)

## Plot a series with the current forecast for that vintage

In [ ]:
fe.plot_vintage(
    data=data,
    variable="cpisa",
    forecast_source=["mpr", "compass conditional", "bvar conditional"],
    frequency="Q",
    vintage_date="2020-03-31",
    metric="yoy",  # Or 'levels', 'yoy' or 'pop'
)

## Hedgehog chart

In [ ]:
fe.plot_hedgehog(
    data=data, variable="cpisa", forecast_source="mpr", metric="yoy", frequency="Q", k=12, convert_to_percentage=True
)

## Plot forecast errors over a rolling window, over time

In [ ]:
data_covid_filtered = data.copy()
data_covid_filtered.filter(custom_filter=fe.covid_filter)
fe.plot_errors_across_time(
    data_covid_filtered,
    variable="gdpkp",
    metric="yoy",
    ma_window=4,
    error="raw",
    sources=["mpr", "baseline ar(p) model"],
    k=12,
    horizons=[0, 4],
)

## Plot forecast errors from a particular vintage

In [ ]:
fe.plot_forecast_errors(
    data=data,
    variable="cpisa",
    metric="yoy",
    frequency="Q",
    source="mpr",
    vintage_date_forecast="2022-03-31",
    k=12,
    convert_to_percentage=True,
)

## Plot average forecast errors

In [ ]:
fe.plot_forecast_errors_by_horizon(
    data=data, variable="cpisa", metric="yoy", frequency="Q", source="mpr", k=12, convert_to_percentage=True
)

## Plot outturn revisions

In [ ]:
fe.plot_outturns(
    data=data,
    variable="gdpkp",
    metric="yoy",
    frequency="Q",
    k=[0, 12],
    fill_k=True,
    convert_to_percentage=True,
    return_plot=False,
)

In [ ]:
fe.plot_outturn_revisions(
    data=data,
    variable="gdpkp",
    metric="yoy",
    frequency="Q",
    k=[4, 12],
    ma_window=4,
    fill_k=True,
    convert_to_percentage=True,
    return_plot=False,
)

# __Forecast Evaluation Methods__

## Accuracy

### Compute accuracy metrics

In [ ]:
accuracy_results = fe.compute_accuracy_statistics(data=data, k=12)

### Plot accuracy metrics using TestResult.plot()

In [ ]:
accuracy_results.plot(variable="cpisa", metric="yoy", frequency="Q", statistic="rmse", convert_to_percentage=True)

### Compare accuracy to benchmark

In [ ]:
# Compare to benchmark and visualize
accuracy_comparison = fe.compare_to_benchmark(
    df=accuracy_results, benchmark_model="baseline ar(p) model", statistic="rmse"
)

# Plot comparison to benchmark
fe.plot_compare_to_benchmark(
    df=accuracy_results,
    variable="cpisa",
    metric="yoy",
    frequency="Q",
    benchmark_model="baseline ar(p) model",
    statistic="rmse",
)

### Comparison table of accuracy metric from all models to a benchmark model

In [ ]:
cpisa_comparison_table = fe.create_comparison_table(
    df=accuracy_results.to_df(),
    variable="cpisa",
    metric="yoy",
    frequency="Q",
    benchmark_model="baseline ar(p) model",
    statistic="rmse",
    horizons=[0, 1, 2, 4, 8, 12],
)

### Run Diebold-Mariano test to investigate relative accuracy formally

In [ ]:
diebold_mariano_results = fe.diebold_mariano_table(
    data=data,
    benchmark_model="mpr",
)

In [ ]:
forecast_data_dm_rolling = data.copy()
forecast_data_dm_rolling.filter(variables=["gdpkp"], metrics=["yoy"], sources=["mpr", "baseline random walk model"])

rolling_dm = fe.rolling_analysis(
    data=forecast_data_dm_rolling,
    window_size=40,
    analysis_func=fe.diebold_mariano_table,
    analysis_args={"benchmark_model": "mpr"},
)

# Use TestResult.plot() - automatically routes to plot_rolling_relative_accuracy
rolling_dm.plot(variable="gdpkp", horizons=[0, 4])

### Run a fluctuation test using the Diebold-Mariano statistic
Fluctuation tests are robust to the repeated nature of rolling tests. The hypothesis of equal average accuracy is rejected if the max statistic over all windows is greater than the flucatuation critical value. See https://doi.org/10.1002/jae.1177.

In [ ]:
rolling_dm_fluctuation = fe.fluctuation_tests(
    data=forecast_data_dm_rolling,
    window_size=40,
    test_func=fe.diebold_mariano_table,
    test_args={"benchmark_model": "mpr"},
)

# Use TestResult.plot() - automatically routes to plot_rolling_relative_accuracy
rolling_dm_fluctuation.plot(variable="gdpkp", horizons=[0, 4])

## Bias

In [ ]:
bias_results = fe.bias_analysis(data=data, source="mpr", k=12, verbose=False)

bias_results.plot(variable="aweagg", source="mpr", metric="yoy", frequency="Q", convert_to_percentage=True)

### Rolling bias analysis

In [ ]:
# Select of subset of results for the rolling analysis
data_gdp = data.copy()
data_gdp.filter(variables=["aweagg"], sources=["mpr"], metrics=["yoy"])

rolling_bias = fe.rolling_analysis(
    data=data_gdp, window_size=40, analysis_func=fe.bias_analysis, analysis_args={"k": 12}
)

rolling_bias.plot(variable="aweagg", source="mpr", horizons=[0, 4, 8, 12])

### Rolling bias analysis with fluctuation test

In [ ]:
rolling_bias_fluctuation = fe.fluctuation_tests(
    data=data_gdp, window_size=40, test_func=fe.bias_analysis, test_args={"k": 12}
)

rolling_bias_fluctuation.plot(horizons=[0, 4, 8, 12])

## Blanchard Leigh

In [ ]:
# Run tests across horizons
bl_results = fe.blanchard_leigh_horizon_analysis(
    data=data,
    source="mpr",
    outcome_variable="cpisa",
    outcome_metric="yoy",
    instrument_variable="gdpkp",
    instrument_metric="yoy",
)

bl_results.plot()

## Weak efficiency (Optimal scaling test)

In [ ]:
weak_efficiency_results = fe.weak_efficiency_analysis(data=data, source="mpr", k=12, verbose=False)

# __Revisions Analysis__

## Testing whether revisions are correlated with forecast errors

In [ ]:
revisions_correlation_results = fe.revisions_errors_correlation_analysis(data=data, source="mpr", k=12)

## Testing whether the final revision can be predicted by past revisions

In [ ]:
revisions_predictable_results = fe.revision_predictability_analysis(data=data, frequency="Q", n_revisions=5)

In [ ]:
fe.plot_average_revision_by_period(data=data, source="mpr", variable="gdpkp", metric="yoy", frequency="Q")

# __Dashboard__

In [ ]:
data.run_dashboard(from_jupyter=True)